## 1. Download and observe dataset

In [2]:
import wget
wget.download(url="https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt")

'input (1).txt'

In [3]:
with open("input.txt", "r", encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters is:", len(text))

length of dataset in characters is: 1115394


In [5]:
# display the first 100 characters
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


## 2. Create Encoder and Decoder by mapping integers to and from characters

In [6]:
# extract all unique characters and sort them
chars = sorted(list(set(text)))
# the amount of unique character is the lenght of the previous list
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# create a mapping from characters to integers
char_to_int = {ch:i for i,ch in enumerate(chars)}
print(char_to_int)

# create a mapping from integers to characters
int_to_char = {i:ch for i,ch in enumerate(chars)}
print(int_to_char)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [8]:
list("stop")

['s', 't', 'o', 'p']

In [9]:
#Encoder: Take string and map it to a list of integers
encode = lambda input_text: [char_to_int[s] for s in list(input_text)]
print(encode("stop"))

# Decoder: take a list of integers, output a string
decode = lambda list_of_ints: "".join([int_to_char[i] for i in list_of_ints])
print(decode(encode("stop")))

[57, 58, 53, 54]
stop


## 3. Split train and test data, create batches

In [10]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [11]:
# Let's now split up the data into train and validation sets
n=int(0.85 *len(data)) # 85 / 15 split
train_data = data[:n]
val_data = data[n:]

In [12]:
block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = train_data[:block_size]
y = train_data[:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input in {context} the target is: {target}")

when input in tensor([18]) the target is: 18
when input in tensor([18, 47]) the target is: 47
when input in tensor([18, 47, 56]) the target is: 56
when input in tensor([18, 47, 56, 57]) the target is: 57
when input in tensor([18, 47, 56, 57, 58]) the target is: 58
when input in tensor([18, 47, 56, 57, 58,  1]) the target is: 1
when input in tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 15
when input in tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 47


In [14]:
torch.manual_seed(1337)
# how many sequences will we process in parallel 
batch_size = 4
# what is the maximum context length for predictions? 
block_size = 8

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)
print("______")

#for each example in a batch
for b in range(batch_size):
    # for every element in an example input
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is  {context.tolist()} the target is: {target}")


inputs:
torch.Size([4, 8])
tensor([[53, 61, 57, 10,  0, 20, 43,  1],
        [39, 41, 43, 42,  1, 58, 46, 43],
        [52, 41, 43,  8,  0,  0, 24, 17],
        [26, 33, 31, 10,  0, 25, 53, 57]])
targets:
torch.Size([4, 8])
tensor([[61, 57, 10,  0, 20, 43,  1, 58],
        [41, 43, 42,  1, 58, 46, 43,  1],
        [41, 43,  8,  0,  0, 24, 17, 27],
        [33, 31, 10,  0, 25, 53, 57, 58]])
______
when input is  [53] the target is: 61
when input is  [53, 61] the target is: 57
when input is  [53, 61, 57] the target is: 10
when input is  [53, 61, 57, 10] the target is: 0
when input is  [53, 61, 57, 10, 0] the target is: 20
when input is  [53, 61, 57, 10, 0, 20] the target is: 43
when input is  [53, 61, 57, 10, 0, 20, 43] the target is: 1
when input is  [53, 61, 57, 10, 0, 20, 43, 1] the target is: 58
when input is  [39] the target is: 41
when input is  [39, 41] the target is: 43
when input is  [39, 41, 43] the target is: 42
when input is  [39, 41, 43, 42] the target is: 1
when input is  [

## 4. Define BigramLanguageModel that trains and predicts sequence of characters

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super(BigramLanguageModel, self).__init__()
        # each token directly reads off the logits for 
        # the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):

        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) 
        if targets is None:
            loss = None
        else:
            #(Batch=batch_size, Time = block_size, Channel = vocab_size)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) #could also use targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is  (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions => self here is the model
            # =>same as doing: logits, loss = m(xb, yb)
            logits, loss = self(idx)
            # focus only on the last time step (that of the T timension)
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.9670, grad_fn=<NllLossBackward0>)


In [16]:
import numpy as np
# Note: expected_cross_entropy_loss
-np.log(1/65) 

4.174387269895637

In [17]:
# Generate from the model
idx = torch.zeros((1, 1), dtype = torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


## 5. Train the model

In [35]:
# create a PyTorch optimizer
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [39]:
batch_size = 32
for steps in range(10_000):
    # sample a new batch of data
    xb, yb = get_batch("train")
    # evaluate the loss
    logits, loss = m(xb, yb)
    # drop previous gradients
    optimizer.zero_grad(set_to_none=True)
    # backpropagate the gradients
    loss.backward()
    # update the gradients
    optimizer.step()
print(loss.item())

2.4408676624298096


In [40]:
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


Wiut;
Winghic
Lot yod aucacat t y hen d s w?
thouryor, che nin me whaineay f I ath s t'ly minos; bye


## 6. Self-attention

In [26]:
# self attention: correlating current token with all previous tokens
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
tril = torch.tril(torch.ones(T,T))
weights = torch.zeros((T,T))
weights = weights.masked_fill(tril ==0, float("-inf"))
weights = F.softmax(weights, dim=-1)
out = weights @ x
#print(tril)
#print(weights)
print(out.shape)

torch.Size([4, 8, 32])


In [35]:
# self attention: correlating current token with all previous tokens
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# introduce self-attentions to allow for non-uniform
# contribution of previous tokens and interesting correlations
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x) # (B, T, 16)
weights = q @ k.transpose(-2, -1)* C**-0.5 # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T,T))
weights = weights.masked_fill(tril ==0, float("-inf"))
weights = F.softmax(weights, dim=-1)
out = weights @ v
#print(tril)
#print(weights)
print(out.shape)

torch.Size([4, 8, 16])


In [36]:
weights

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5908, 0.4092, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2635, 0.3681, 0.3684, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2168, 0.2405, 0.2929, 0.2498, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2132, 0.1782, 0.2796, 0.1329, 0.1961, 0.0000, 0.0000, 0.0000],
         [0.1671, 0.1776, 0.1480, 0.1278, 0.1718, 0.2076, 0.0000, 0.0000],
         [0.1459, 0.1268, 0.2029, 0.1297, 0.1253, 0.1327, 0.1368, 0.0000],
         [0.1318, 0.1365, 0.1477, 0.1305, 0.1331, 0.0771, 0.1023, 0.1409]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4392, 0.5608, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4208, 0.2937, 0.2855, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2053, 0.2712, 0.2246, 0.2988, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2265, 0.1420, 0.2498, 0.1797, 0.2020, 0.0000, 0.0000, 0.0000],
         [0.2402, 0.108